<a href="https://colab.research.google.com/github/PranavViswanath/Data-Science/blob/main/Projects/HTGC_Kitchen_2021/Estimating_Demand_Calculator_PranavV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ATTENTION: please run ALL of the cells before calling the functions below- choose the latest cleaned touchbistro data when prompted by the first cell, then run each cell afterwards and
# wait for the green check mark before moving on from each one
# 8/19- 4 criteria: avg day 7/8 pm, avg day + less than median, avg top x %ile hours for any stockouts, voids
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
import scipy.stats as stats
import datetime as dt
import warnings
import statsmodels.api as sm
uploaded = files.upload()
import io
df = pd.read_excel(io.BytesIO(uploaded['Touchbistro data cleaned 080221.xlsx']))
df['Paid Time Rounded'] = df['Paid Time'].dt.round('60min')
df['Dates'] = pd.to_datetime(df['Paid Time Rounded']).dt.date
df['Time'] = pd.to_datetime(df['Paid Time Rounded']).dt.time
noncovid = (df['Dates']<pd.to_datetime('2020-03-14')) | (df['Dates']>pd.to_datetime('2021-2-28'))
noncovid = df.loc[noncovid]
n2019 = (df['Dates']>pd.to_datetime('2019-03-29')) & (df['Dates']<pd.to_datetime('2020-03-14'))
n2019 = df.loc[n2019]

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Saving Touchbistro data cleaned 080221.xlsx to Touchbistro data cleaned 080221.xlsx


In [ ]:
def demand_calculator_avgday(item, day, frame):
  arr=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
  pd.set_option('mode.chained_assignment', None)
  new = frame[(frame['Day']==day) & (frame['Rev-Menu itemName']==item)]
  pivot_hourly = pd.pivot_table(new, index='Dates', columns='Time', values='Quantity', aggfunc='sum', margins=True, margins_name='Total')
  flattened = pd.DataFrame(pivot_hourly.to_records())
  flattened['status'] = np.where(np.isnan(flattened['19:00:00']), 'Out', 'In')
  flattened.sort_values('Total')
  temp = flattened[['Total', 'status', 'Dates']]
  temp['ln'] = np.log(temp['Total'])
  temp = temp.sort_values('status')
  temp=temp.drop(temp['Total'].idxmax())
  new = temp.sort_values(['Total', 'ln'])
  new=new.reset_index(drop=True)
  ###########
  i=1
  a=[]
  while(i<new.shape[0]+1):
    a.append(i)
    i+=1
  new.index=a
  ###############
  rows=new.shape[0]+1
  new['perc'] = 1-abs((new.index-rows)/rows)
  import scipy.stats as stats
  new['z'] = stats.norm.ppf(new['perc'])
  instock = new.loc[new['status']=='In']
  outstock = new.loc[new['status']=='Out']
  m, b = np.polyfit(instock['z'], instock['Total'], 1)
  new["imputed"] = np.where(new['status']=='Out' , ((new['z']*m)+b), 0)
  new['final_y'] = np.where(new['imputed']==0, new['Total'], new['imputed'])
  instockz = new.loc[(new['status']=='In') & (new['imputed']==0)]
  outstockz = new.loc[new['status']=='Out']
  ###############
  a, c = np.polyfit(instock['z'], instock['ln'], 1)
  new["imputed_ln"] = np.where(new['status']=='Out' , ((new['z']*a)+c), 0)
  new['final_ln'] = np.where(new['imputed_ln']==0, new['ln'], new['imputed_ln'])
  new['final_y_ln'] = np.exp(new['final_ln'])
  instockz = new.loc[(new['status']=='In') & (new['imputed']==0)]
  outstockz = new.loc[new['status']=='Out']
  outstock['imputed'] = new['imputed']
  outstock['final_y_ln'] = new['final_y_ln']
  '''
  ####################
  plt.scatter(instockz['z'], instockz['Total'], color='b', label='In Stock')
  plt.scatter(outstockz['z'], outstockz['Total'], color='r', label='Out of Stock')
  plt.scatter(outstockz['z'], outstockz['imputed'], color='g', label='Imputed')
  plt.title('In/Out + Imputed: Normal Average Day')
  plt.legend(loc="upper left")
  plt.show()
  ####################
  plt.scatter(instockz['z'], instockz['Total'], color='b', label='In Stock')
  plt.scatter(outstockz['z'], outstockz['Total'], color='r', label='Out of Stock')
  plt.scatter(outstockz['z'], outstockz['final_y_ln'], color='g', label='Imputed')
  plt.title('In/Out + Imputed: LogNormal Average Day')
  plt.legend(loc="upper left")
  plt.show()
  '''
  ######################
  import statsmodels.api as sm
  x=new['z']
  y=new['final_y']
  x=sm.add_constant(x)
  model = sm.OLS(endog=y, exog=x)
  results = model.fit()
  #print('Normal Results: ' + str(results.rsquared) + ' , ' + str(results.fvalue) + ' , ' + str(results.pvalues))
  print('Average sales for ' + item + ' on ' + arr[day-1] + ': ' + str(new['Total'].mean()))
  #print(' Average instock sales: '+ str(instockz['Total'].mean()))
  #print('Estimated Mean Demand For Normal: ' + str(new['final_y'].mean()))
  z=new['z']
  m=new['final_y_ln']
  z=sm.add_constant(z)
  model = sm.OLS(endog=m, exog=z)
  result = model.fit()
  #print('LogNormal Results: ' + str(result.rsquared) + ' , ' + str(result.fvalue) + ' , ' + str(result.pvalues))
  print('Estimated demand for ' + item + ' on ' + arr[day-1] + ': ' + str(new['final_y_ln'].mean()))
  #print('LSE Normal Stats: Number of Sales: ' + str(new.shape[0]) +', Average all sales: '+ str(new['Total'].mean()) + ', Number of Instock: ' + str(instockz.shape[0]) +', Mean of instock: '+ str(instockz['Total'].mean()) +', Variance of instock: ' +str(instockz.var()['Total']))
  #print('LSE LogNormal Stats: Number of Sales: ' + str(new.shape[0]) +', Average all sales: '+ str(new['Total'].mean()) + ', Number of Instock: ' + str(instockz.shape[0]) +', Mean of instock: '+ str(instockz['ln'].mean()) +', Variance of instock: ' +str(instockz.var()['ln']))
#'''

In [ ]:
demand_calculator_avgday('Masala Dosa', 7, n2019)

Average sales for Masala Dosa on Saturday: 171.81632653061226
Estimated demand for Masala Dosa on Saturday: 173.68877143532146


In [ ]:
def demand_calculator_avgday_median(item, day, frame):
  pd.set_option('mode.chained_assignment', None)
  new = frame[(frame['Day']==day) & (frame['Rev-Menu itemName']==item)]
  pivot_hourly = pd.pivot_table(new, index='Dates', columns='Time', values='Quantity', aggfunc='sum', margins=True, margins_name='Total')
  flattened = pd.DataFrame(pivot_hourly.to_records())
  med=flattened['Total'].median()
  flattened['status'] = np.where((np.isnan(flattened['19:00:00'])&(flattened['Total']<med)), 'Out', 'In')
  flattened.sort_values('Total')
  temp = flattened[['Total', 'status', 'Dates']]
  temp['ln'] = np.log(temp['Total'])
  temp = temp.sort_values('status')
  temp=temp.drop(temp['Total'].idxmax())
  new = temp.sort_values(['Total', 'ln'])
  new=new.reset_index(drop=True)
  ###########
  i=1
  a=[]
  while(i<new.shape[0]+1):
    a.append(i)
    i+=1
  new.index=a
  ###############
  rows=new.shape[0]+1
  new['perc'] = 1-abs((new.index-rows)/rows)
  new['z'] = stats.norm.ppf(new['perc'])
  instock = new.loc[new['status']=='In']
  outstock = new.loc[new['status']=='Out']
  m, b = np.polyfit(instock['z'], instock['Total'], 1)
  new["imputed"] = np.where(new['status']=='Out' , ((new['z']*m)+b), 0)
  new['final_y'] = np.where(new['imputed']==0, new['Total'], new['imputed'])
  instockz = new.loc[(new['status']=='In') & (new['imputed']==0)]
  outstockz = new.loc[new['status']=='Out']
  ###############
  a, c = np.polyfit(instock['z'], instock['ln'], 1)
  new["imputed_ln"] = np.where(new['status']=='Out' , ((new['z']*a)+c), 0)
  new['final_ln'] = np.where(new['imputed_ln']==0, new['ln'], new['imputed_ln'])
  new['final_y_ln'] = np.exp(new['final_ln'])
  instockz = new.loc[(new['status']=='In') & (new['imputed']==0)]
  outstockz = new.loc[new['status']=='Out']
  outstock['imputed'] = new['imputed']
  outstock['final_y_ln'] = new['final_y_ln']
  ####################
  plt.scatter(instockz['z'], instockz['Total'], color='b', label='In Stock')
  plt.scatter(outstockz['z'], outstockz['Total'], color='r', label='Out of Stock')
  plt.scatter(outstockz['z'], outstockz['imputed'], color='g', label='Imputed')
  plt.title('In/Out + Imputed: Normal Average Day')
  plt.legend(loc="upper left")
  plt.show()
  ####################
  plt.scatter(instockz['z'], instockz['Total'], color='b', label='In Stock')
  plt.scatter(outstockz['z'], outstockz['Total'], color='r', label='Out of Stock')
  plt.scatter(outstockz['z'], outstockz['final_y_ln'], color='g', label='Imputed')
  plt.title('In/Out + Imputed: LogNormal Average Day')
  plt.legend(loc="upper left")
  plt.show()
  ######################
  x=new['z']
  y=new['final_y']
  x=sm.add_constant(x)
  model = sm.OLS(endog=y, exog=x)
  results = model.fit()
  #print('Normal Results: ' + str(results.rsquared) + ' , ' + str(results.fvalue) + ' , ' + str(results.pvalues))
  print('Average all sales: '+ str(new['Total'].mean()) + ' Average instock sales: '+ str(instockz['Total'].mean()))
  print('Estimated Mean Demand For Normal: ' + str(new['final_y'].mean()))
  z=new['z']
  m=new['final_y_ln']
  z=sm.add_constant(z)
  model = sm.OLS(endog=m, exog=z)
  result = model.fit()
  #print('LogNormal Results: ' + str(result.rsquared) + ' , ' + str(result.fvalue) + ' , ' + str(result.pvalues))
  print('Estimated Mean Demand For LogNormal: ' + str(new['final_y_ln'].mean()))
  #print('LSE Normal Stats: Number of Sales: ' + str(new.shape[0]) +', Average all sales: '+ str(new['Total'].mean()) + ', Number of Instock: ' + str(instockz.shape[0]) +', Mean of instock: '+ str(instockz['Total'].mean()) +', Variance of instock: ' +str(instockz.var()['Total']))
  #print('LSE LogNormal Stats: Number of Sales: ' + str(new.shape[0]) +', Average all sales: '+ str(new['Total'].mean()) + ', Number of Instock: ' + str(instockz.shape[0]) +', Mean of instock: '+ str(instockz['ln'].mean()) +', Variance of instock: ' +str(instockz.var()['ln']))

In [ ]:
def demand_calculator_avg_hourly_percentile(item, day, frame, x):
  pd.set_option('mode.chained_assignment', None)
  new = frame[(frame['Day']==day) & (frame['Rev-Menu itemName']==item)]
  pivot_hourly = pd.pivot_table(new, index='Dates', columns='Time', values='Quantity', aggfunc=('sum'), margins=True, margins_name='Total')
  flattened = pd.DataFrame(pivot_hourly.to_records())
  flattened = flattened.mean(axis=0, skipna=True).rename('Mean').pipe(flattened.append)
  rows=(flattened.shape[0])-1
  arr=flattened.iloc[rows].to_numpy().tolist()
  arr.sort()
  arr.remove(max(arr)) #total
  arr = arr[~pd.isnull(arr)]
  tile=stats.rankdata(arr, "average")/len(arr)
  greater=0
  for i in tile:
    if i>x:
      greater+=1
  arr=arr[greater:]
  flattened.drop(['Dates'], axis=1, inplace=True)
########## finding top x percent range of hours
  finrow=[]
  for i in flattened:
    boo=[]
    for r in arr:
      item=flattened[i]
      if item.iloc[-1] == r:
        z=True
      else:
        z=False
      boo.append(z)
    if True in boo:
      finrow.append(True)
    else:
      finrow.append(False)
############   final table with only selected hours   
  falsecols = [i for i in range(len(finrow)) if finrow[i]==False]
  falsecols.remove(max(falsecols))
  flattened=flattened.drop(flattened.columns[falsecols], axis=1)
#################### criteria
  def status(flattened):
    if (flattened[:]).isnull().values.any():
      return 'Out'
    else:
      return 'In'
  flattened['status'] = flattened.apply(status, axis=1)
  flattened.sort_values('Total')
  temp = flattened[['Total', 'status']]
  temp['ln'] = np.log(temp['Total'])
  temp = temp.sort_values('status')
  temp=temp.drop(temp['Total'].idxmax())
  new = temp.sort_values(['Total', 'ln'])
  new=new.drop('Mean')
  new=new.reset_index(drop=True)
  ###########
  i=1
  a=[]
  while(i<new.shape[0]+1):
    a.append(i)
    i+=1
  new.index=a
  ###############
  rows=new.shape[0]+1
  new['perc'] = 1-abs((new.index-rows)/rows)
  new['z'] = stats.norm.ppf(new['perc'])
  instock = new.loc[new['status']=='In']
  outstock = new.loc[new['status']=='Out']
  m, b = np.polyfit(instock['z'], instock['Total'], 1)
  new["imputed"] = np.where(new['status']=='Out' , ((new['z']*m)+b), 0)
  new['final_y'] = np.where(new['imputed']==0, new['Total'], new['imputed'])
  instockz = new.loc[(new['status']=='In') & (new['imputed']==0)]
  outstockz = new.loc[new['status']=='Out']
  ###############
  a, c = np.polyfit(instock['z'], instock['ln'], 1)
  new["imputed_ln"] = np.where(new['status']=='Out' , ((new['z']*a)+c), 0)
  new['final_ln'] = np.where(new['imputed_ln']==0, new['ln'], new['imputed_ln'])
  new['final_y_ln'] = np.exp(new['final_ln'])
  instockz = new.loc[(new['status']=='In') & (new['imputed']==0)]
  outstockz = new.loc[new['status']=='Out']
  outstock['imputed'] = new['imputed']
  outstock['final_y_ln'] = new['final_y_ln']
  ####################
  plt.scatter(instockz['z'], instockz['Total'], color='b', label='In Stock')
  plt.scatter(outstockz['z'], outstockz['Total'], color='r', label='Out of Stock')
  plt.scatter(outstockz['z'], outstockz['imputed'], color='g', label='Imputed')
  plt.title('In/Out + Imputed: Normal Average Day')
  plt.legend(loc="upper left")
  plt.show()
  ####################
  plt.scatter(instockz['z'], instockz['Total'], color='b', label='In Stock')
  plt.scatter(outstockz['z'], outstockz['Total'], color='r', label='Out of Stock')
  plt.scatter(outstockz['z'], outstockz['final_y_ln'], color='g', label='Imputed')
  plt.title('In/Out + Imputed: LogNormal Average Day')
  plt.legend(loc="upper left")
  plt.show()
  ######################
  x=new['z']
  y=new['final_y']
  x=sm.add_constant(x)
  model = sm.OLS(endog=y, exog=x)
  results = model.fit()
  #print('Normal Results: ' + str(results.rsquared) + ' , ' + str(results.fvalue) + ' , ' + str(results.pvalues))
  print('Average all sales: '+ str(new['Total'].mean()) + ' Average instock sales: '+ str(instockz['Total'].mean()))
  print('Estimated Mean Demand For Normal: ' + str(new['final_y'].mean()))
  z=new['z']
  m=new['final_y_ln']
  z=sm.add_constant(z)
  model = sm.OLS(endog=m, exog=z)
  result = model.fit()
  #print('LogNormal Results: ' + str(result.rsquared) + ' , ' + str(result.fvalue) + ' , ' + str(result.pvalues))
  print('Estimated Mean Demand For LogNormal: ' + str(new['final_y_ln'].mean()))
  #print('LSE Normal Stats: Number of Sales: ' + str(new.shape[0]) +', Average all sales: '+ str(new['Total'].mean()) + ', Number of Instock: ' + str(instockz.shape[0]) +', Mean of instock: '+ str(instockz['Total'].mean()) +', Variance of instock: ' +str(instockz.var()['Total']))
  #print('LSE LogNormal Stats: Number of Sales: ' + str(new.shape[0]) +', Average all sales: '+ str(new['Total'].mean()) + ', Number of Instock: ' + str(instockz.shape[0]) +', Mean of instock: '+ str(instockz['ln'].mean()) +', Variance of instock: ' +str(instockz.var()['ln']))
 

In [ ]:
demand_calculator_avg_hourly_percentile('Masala Dosa', 7, df, .25)

TypeError: ignored

In [ ]:
def demand_calculator_void(item, day, frame):
  pd.set_option('mode.chained_assignment', None)
  new = frame[(frame['Day']==day) & (frame['Rev-Menu itemName']==item)]
  pivot_hourly = pd.pivot_table(new, index='Dates', values=['Quantity', 'Is Voided'], aggfunc='sum', margins=True, margins_name='Total')
  flattened = pd.DataFrame(pivot_hourly.to_records())
  flattened['status'] = np.where(flattened['Is Voided']>0, 'Out', 'In')
  flattened.sort_values('Quantity')
  temp = flattened[['Quantity', 'status', 'Dates']]
  temp['ln'] = np.log(temp['Quantity'])
  temp = temp.sort_values('status')
  temp.drop(temp['Quantity'].idxmax())
  new = temp.sort_values(['Quantity', 'ln'])
  new=new.reset_index(drop=True)
  print(new)
  #############
  i=1
  a=[]
  while(i<new.shape[0]+1):
    a.append(i)
    i+=1
  new.index=a
############## 
  new=new.drop(new['Quantity'].idxmax())
  rows=new.shape[0]+1
  new['perc'] = 1-abs((new.index-rows)/rows)
  new['z'] = stats.norm.ppf(new['perc'])
  instock = new.loc[new['status']=='In']
  outstock = new.loc[new['status']=='Out']
  m, b = np.polyfit(instock['z'], instock['Quantity'], 1)
  new["imputed"] = np.where(new['status']=='Out' , ((new['z']*m)+b), 0)
  new['final_y'] = np.where(new['imputed']==0, new['Quantity'], new['imputed'])
  instockz = new.loc[(new['status']=='In') & (new['imputed']==0)]
  outstockz = new.loc[new['status']=='Out']
  ###############
  a, c = np.polyfit(instock['z'], instock['ln'], 1)
  new["imputed_ln"] = np.where(new['status']=='Out' , ((new['z']*a)+c), 0)
  new['final_ln'] = np.where(new['imputed_ln']==0, new['ln'], new['imputed_ln'])
  new['final_y_ln'] = np.exp(new['final_ln'])
  new = new.drop(new['Quantity'].idxmax())
  instockz = new.loc[(new['status']=='In') & (new['imputed']==0)]
  outstockz = new.loc[new['status']=='Out']
  outstock['imputed'] = new['imputed']
  outstock['final_y_ln'] = new['final_y_ln']
  ####################
  outstock=outstock.drop(outstock['Quantity'].idxmax())
  
  #################
  plt.scatter(instockz['z'], instock['Quantity'], color='b', label='In Stock')
  outstockz['bad'] = outstock['Quantity']
  outstockz['imputed'] = outstock['imputed']
  plt.scatter(outstockz['z'], outstockz['bad'], color='r', label='Out of Stock')
  plt.scatter(outstockz['z'], outstockz['imputed'], color='g', label='Imputed')
  plt.title('In/Out + Imputed: Normal Void')
  plt.legend(loc="upper left")
  plt.show()
  ####################
  plt.scatter(instockz['z'], instock['Quantity'], color='b', label='In Stock')
  outstockz['bad'] = outstock['Quantity']
  outstockz['final_y_ln'] = outstock['final_y_ln']
  outstockz = outstockz.drop(outstockz['Quantity'].idxmax())
  plt.scatter(outstockz['z'], outstockz['Quantity'], color='r', label='Out of Stock')
  plt.scatter(outstockz['z'], outstockz['final_y_ln'], color='g', label='Imputed')
  plt.title('In/Out + Imputed: LogNormal Void')
  plt.legend(loc="upper left")
  plt.show()
  ######################
  import statsmodels.api as sm
  x=new['z']
  y=new['final_y']
  x=sm.add_constant(x)
  model = sm.OLS(endog=y, exog=x)
  results = model.fit()
  #print('Normal Results: ' + str(results.rsquared) + ' , ' + str(results.fvalue) + ' , ' + str(results.pvalues))
  print('Average all sales: '+ str(new['Quantity'].mean()) + ' Average instock sales: '+ str(instockz['Quantity'].mean()))
  print('Estimated Mean Demand For Normal: ' + str(new['final_y'].mean()))
  z=new['z']
  m=new['final_y_ln']
  z=sm.add_constant(z)
  model = sm.OLS(endog=m, exog=z)
  result = model.fit()
  #print('LogNormal Results: ' + str(result.rsquared) + ' , ' + str(result.fvalue) + ' , ' + str(result.pvalues))
  print('Estimated Mean Demand For LogNormal: ' + str(new['final_y_ln'].mean()))
  print('Median: ' + str(new['Quantity'].median()))
  #sales+demand for all and instock
  #print('LSE Normal Stats: Number of Sales: ' + str(new.shape[0]) +', Average all sales: '+ str(new['Quantity'].mean()) + ', Number of Instock: ' + str(instockz.shape[0]) +', Mean of instock: '+ str(instockz['Quantity'].mean()) +', Variance of instock: ' +str(instockz.var()['Quantity']))
  #print('LSE LogNormal Stats: Number of Sales: ' + str(new.shape[0]) +', Average all sales: '+ str(new['Quantity'].mean()) + ', Number of Instock: ' + str(instockz.shape[0]) +', Mean of instock: '+ str(instockz['ln'].mean()) +', Variance of instock: ' +str(instockz.var()['ln']))